In [ ]:
#nested cross-validation

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, GroupKFold, cross_val_score
from sklearn.metrics import classification_report

from scipy.stats import binomtest

import seaborn as sns; sns.set(font_scale=1.2)

In [12]:
#import data
all_df = pd.read_csv('processed_data/all_faces_coded.csv')

#iteration elite face sample
elite_df = all_df[all_df['eliteSelected']==1]

#selected face sample
selected_df = all_df[all_df['faceSelected']==1]

#filter patient and control samples
PT_df=elite_df[elite_df['Group']=='PT']
HC_df=elite_df[elite_df['Group']=='HC']

print(elite_df.groupby('Group')['ID'].count())
print(selected_df.groupby('Group')['ID'].count())

Group
HC    504
PT    504
Name: ID, dtype: int64
Group
HC    2010
PT    1793
Name: ID, dtype: int64


In [18]:
#assign features, labels and subject group variable
X = elite_df.iloc[:,1:150]
y = elite_df['Group']
Group = elite_df['ID']

In [43]:
# configure the cross-validation procedure
cv_inner = GroupKFold(n_splits=5)
# define the model
model = SVC(random_state=102)
# cross-validation metrics
params={ 
    'kernel': ['rbf'],
    'C': [.1,.5,1,10,30,40,50,75,100,500,1000],
    'gamma': [.001,.005,.01,.05,.07,.1,.5,1,5,10,50]
}
# define search
search = GridSearchCV(model, params, cv=cv_inner)
# configure the cross-validation procedure
cv_outer = GroupKFold(n_splits=10)
# execute the nested cross-validation
scores = cross_val_score(estimator=search, X=X, y=y, groups=Group, cv=cv_outer, fit_params={'groups': Group}, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

NameError: name 'mean' is not defined

In [45]:
scores

array([0.66666667, 0.5       , 0.57291667, 0.63541667, 0.71875   ,
       0.53125   , 0.44791667, 0.38541667, 0.60416667, 0.34375   ])